In [1]:
import pandas as pd
import pickle
from colormap import hex2rgb
import numpy as np
from collections import Counter, defaultdict
from tqdm.notebook import trange, tqdm
from sklearn.cluster import AgglomerativeClustering

# Split into columns

In [30]:
df = pd.read_csv("./FinalDataSet.csv")
df = df.drop(["Unnamed: 0"],axis = 1)
#df.to_csv("neo4j_data.csv")

In [8]:
with open('./all_artwork_palettes.pickle','rb') as pickle_file:
    content = pickle.load(pickle_file)

In [9]:
color_list = []
for id in content.keys():
    color_list.append([id]+list(content[id]['c_code'])[:4])

In [10]:
color_df = pd.DataFrame(color_list, columns = ['id', 'color1', 'color2','color3','color4'])
color_df.to_csv("ColorPaletteData.csv")

In [11]:
color_df

,id,color1,color2,color3,color4
0,photography560,#FFFFFF,#6E6F69,#DAD4BE,#9A968A
1,mixed-media1297,#151515,#9BC53F,#E5E5E3,#C74138
2,photography574,#F8F8F8,#949399,#D1D1D1,#68696E
3,prints1038,#FFFFFF,#C5963E,#E6E5A5,#E4B4B0
4,works-on-paper1080,#597739,#4F6942,#595731,#3B533D
...,...,...,...,...,...
2028,photography1214,#E0F3FA,#181913,#38424E,#575852
2029,painting534,#FAFAFA,#252324,#CEB000,#FFEA00
2030,prints1021,#EBECE6,#F5AC8C,#F05B33,#900A25
2031,works-on-paper977,#F6E6C2,#EBEFF0,#777753,#F9DF92


In [18]:
df

,id,media,url,artist_name,artist_url,work_name,work_year,media_long,dimension,gallery_name,...,bid,price,img_url,artist_bio,artist_website,artist_ins,color1,color2,color3,color4
0,painting15,painting,https://www.artsy.net/artwork/hebru-brantley-d...,Hebru Brantley,https://www.artsy.net/artist/hebru-brantley,"Death Of The Black Fairy, The Great Debate Pt. 5",2020,Acrylic & Diamond Dust on Canvas,60 × 60 in | 152.4 × 152.4 cm,Corridor Contemporary,...,False,125000.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,"Hebru Brantley, a Bronzeville-native living an...",www.hebrubrantley.com,https://www.instagram.com/hebrubrantley/?hl=en,#E7DFCC,#A69784,#403931,#877C76
1,painting37,painting,https://www.artsy.net/artwork/miaz-brothers-ma...,Miaz Brothers,https://www.artsy.net/artist/miaz-brothers,Maestro,2022,Acrylic on canvas,40 1/5 × 29 9/10 in | 102 × 76 cm,Maddox Gallery,...,False,17500.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,The Miaz Brothers present a radical new take o...,www.miazbrothers.com,https://www.instagram.com/miazbrothers/?hl=en,#181A15,#2F342E,#43492F,#B8A36C
2,painting41,painting,https://www.artsy.net/artwork/sandra-chevrier-...,Sandra Chevrier,https://www.artsy.net/artist/sandra-chevrier,La cage où le brouillard traverse l’esprit,2017,Oil on canvas,12 × 12 in | 30.5 × 30.5 cm,West Chelsea Contemporary,...,False,5000.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,Sandra Chevrier is a Canadian contemporary art...,www.sandrachevrier.com,https://www.instagram.com/sandrachevrier/?hl=en,#7F7262,#9E9181,#FFFFFF,#D2C9B8
3,painting45,painting,https://www.artsy.net/artwork/florian-eymann-b...,Florian Eymann,https://www.artsy.net/artist/florian-eymann,BAYC Clueless Sailor,2022,Oil on canvas,43 × 34 1/2 in | 109.2 × 87.6 cm,Avant Gallery,...,False,11000.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,Florian Eymann is a self-taught French artist ...,www.florianeymann.com,https://www.instagram.com/florianeymann/?hl=en,#191816,#B8B3A0,#4E443A,#857C6B
4,painting52,painting,https://www.artsy.net/artwork/miaz-brothers-a-...,Miaz Brothers,https://www.artsy.net/artist/miaz-brothers,A Promising Young Man,2022,Acrylic on canvas,48 × 35 4/5 in | 122 × 91 cm,Maddox Gallery,...,False,17500.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,The Miaz Brothers present a radical new take o...,www.miazbrothers.com,https://www.instagram.com/miazbrothers/?hl=en,#13140F,#5B4329,#2C2D27,#A47C41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522,design1454,design,https://www.artsy.net/artwork/ed-ruscha-he-up-...,Ed Ruscha,https://www.artsy.net/artist/ed-ruscha,He Up and Went Downtown - Plate,2020,Porcelain plate,10 1/2 × 10 1/2 in | 26.7 × 26.7 cm,West Chelsea Contemporary,...,False,3144.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,Edward Ruscha is an American artist from the P...,edruscha.com,NaN,NaN,NaN,NaN,NaN
2523,design1463,design,https://www.artsy.net/artwork/zhoujie-zhang-br...,Zhoujie Zhang,https://www.artsy.net/artist/zhoujie-zhang,Brass Bowtie Chair (SQN1-F2A),2014,Brass,29 1/2 × 23 3/5 × 23 3/5 in | 75 × 60 × 60 cm,Gallery ALL,...,False,12500.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,"Zhoujie Zhang is a Chinese furniture designer,...",www.zhangzhoujie.com,NaN,NaN,NaN,NaN,NaN
2524,design1467,design,https://www.artsy.net/artwork/jeff-koons-lips-16,Jeff Koons,https://www.artsy.net/artist/jeff-koons,Lips,2012,"Porcelain, white, screenprint",12 1/5 × 12 1/5 in | 31 × 31 cm,Almine Rech,...,False,728.0,https://d7hftxdivxxvm.cloudfront.net?resize_to...,Jeff Koons is an American artist known for his...,www.jeffkoons.com,https://www.instagram.com/jeffkoons/?hl=en,NaN,NaN,NaN,NaN
2525,design1481,design,https://www.artsy.net/artwork/pablo-picasso-ma...,Pablo Picasso,https://www.artsy.net/artist/pablo-picasso,Madoura Ceramic Pitcher 'Yan Soleil' Ramié 516,1963,"Ceramic, Terracotta",7 in diameter | 17.8 cm diameter,Hirth Fine Art,...,False,15000.0,https://d7hftxdivxxvm.cl

# Add color group

In [12]:
colors = color_df.iloc[:, -4:]
all_colors = [a for l in [colors[c].unique() for c in colors.columns] for a in l if isinstance(a, str)]
print("total number of hexcodes: ", len(all_colors))
all_colors = list(set(all_colors))
print("total number of unique hexcodes: ", len(all_colors))

total number of hexcodes:  6725
total number of unique hexcodes:  6230


In [17]:
colors.head()

,color1,color2,color3,color4
0,#FFFFFF,#6E6F69,#DAD4BE,#9A968A
1,#151515,#9BC53F,#E5E5E3,#C74138
2,#F8F8F8,#949399,#D1D1D1,#68696E
3,#FFFFFF,#C5963E,#E6E5A5,#E4B4B0
4,#597739,#4F6942,#595731,#3B533D


In [13]:
def dist(hex1, hex2):
    hex1 = hex1.lstrip("#")
    hex2 = hex2.lstrip("#")
    #get red/green/blue int values of hex1
    r1, g1, b1 = tuple(int(hex1[i:i+2], 16) for i in (0, 2, 4))
    # get red/green/blue int values of hex2
    r2, g2, b2 = tuple(int(hex2[i:i+2], 16) for i in (0, 2, 4))
    # calculate differences between reds, greens and blues
    r = 255 - abs(r1 - r2);
    g = 255 - abs(g1 - g2);
    b = 255 - abs(b1 - b2);
    # limit differences between 0 and 1
    r /= 255;
    g /= 255;
    b /= 255;
    # 0 means opposite colors, 1 means same colors
    return (r + g + b) / 3;


In [14]:
N = len(all_colors)

pdist = np.ones((N, N)) # initialize as ones because each color to itself should have similarity = 1

for i in tqdm(range(N)):
    for j in range(i + 1, N):
        if isinstance(all_colors[i], str) and isinstance(all_colors[j], str):
            pdist[i,j] = dist(all_colors[i], all_colors[j])
            pdist[j,i] = pdist[i,j]

  0%|          | 0/6230 [00:00<?, ?it/s]

In [15]:
clustering = AgglomerativeClustering(n_clusters=300, affinity='precomputed', linkage='complete').fit(pdist)

In [16]:
color_groups = defaultdict(list)
for idx, label in enumerate(clustering.labels_):
    color_groups[label].append(all_colors[idx])

In [19]:
code_to_group = dict()
for group_id, codes in color_groups.items():
    for code in codes:
        code_to_group[code] = group_id

In [20]:
groups = []
for _, c1, c2, c3, c4 in colors.itertuples():
    if isinstance(c1, str) and isinstance(c2, str) and isinstance(c3, str) and isinstance(c4, str):
        g1 = code_to_group[c1]
        g2 = code_to_group[c2]
        g3 = code_to_group[c3]
        g4 = code_to_group[c4]
        groups.append([g1, g2, g3, g4])
    else:
        groups.append([-1, -1, -1, -1])
    

In [21]:
color_group_df = pd.DataFrame(groups)
color_group_df.columns = ['color_group1', 'color_group2', 'color_group3', 'color_group4']

In [23]:
color_group_df

,color_group1,color_group2,color_group3,color_group4
0,33,193,81,78
1,140,185,279,167
2,289,101,166,73
3,33,12,129,297
4,229,110,248,246
...,...,...,...,...
2028,191,35,18,29
2029,131,90,185,13
2030,200,293,217,120
2031,276,245,256,83


In [24]:
color_df = pd.concat([color_df, color_group_df], axis=1)

In [32]:
color_df.columns

Index(['id', 'color1', 'color2', 'color3', 'color4', 'color_group1',
       'color_group2', 'color_group3', 'color_group4'],
      dtype='object')

# Merge with full dataset 

In [33]:
df.columns

Index(['id', 'media', 'url', 'artist_name', 'artist_url', 'work_name',
       'work_year', 'media_long', 'dimension', 'gallery_name', 'gallery_url',
       'bid', 'price', 'img_url', 'artist_bio', 'artist_website', 'artist_ins',
       'color1', 'color2', 'color3', 'color4'],
      dtype='object')

In [37]:
df = df.merge(color_df, how="left", on = "id")

KeyError: 'id'

In [29]:
df.columns

Index(['id', 'media', 'url', 'artist_name', 'artist_url', 'work_name',
       'work_year', 'media_long', 'dimension', 'gallery_name', 'gallery_url',
       'bid', 'price', 'img_url', 'artist_bio', 'artist_website', 'artist_ins',
       'color1_x', 'color2_x', 'color3_x', 'color4_x', 'color1_y', 'color2_y',
       'color3_y', 'color4_y', 'color_group1', 'color_group2', 'color_group3',
       'color_group4'],
      dtype='object')

In [26]:
df.head()

,id,media,url,artist_name,artist_url,work_name,work_year,media_long,dimension,gallery_name,...,color3_x,color4_x,color1_y,color2_y,color3_y,color4_y,color_group1,color_group2,color_group3,color_group4
0,painting15,painting,https://www.artsy.net/artwork/hebru-brantley-d...,Hebru Brantley,https://www.artsy.net/artist/hebru-brantley,"Death Of The Black Fairy, The Great Debate Pt. 5",2020,Acrylic & Diamond Dust on Canvas,60 × 60 in | 152.4 × 152.4 cm,Corridor Contemporary,...,#403931,#877C76,#E7DFCC,#A69784,#403931,#877C76,229.0,31.0,218.0,28.0
1,painting37,painting,https://www.artsy.net/artwork/miaz-brothers-ma...,Miaz Brothers,https://www.artsy.net/artist/miaz-brothers,Maestro,2022,Acrylic on canvas,40 1/5 × 29 9/10 in | 102 × 76 cm,Maddox Gallery,...,#43492F,#B8A36C,#181A15,#2F342E,#43492F,#B8A36C,166.0,189.0,29.0,7.0
2,painting41,painting,https://www.artsy.net/artwork/sandra-chevrier-...,Sandra Chevrier,https://www.artsy.net/artist/sandra-chevrier,La cage où le brouillard traverse l’esprit,2017,Oil on canvas,12 × 12 in | 30.5 × 30.5 cm,West Chelsea Contemporary,...,#FFFFFF,#D2C9B8,#7F7262,#9E9181,#FFFFFF,#D2C9B8,197.0,111.0,33.0,54.0
3,painting45,painting,https://www.artsy.net/artwork/florian-eymann-b...,Florian Eymann,https://www.artsy.net/artist/florian-eymann,BAYC Clueless Sailor,2022,Oil on canvas,43 × 34 1/2 in | 109.2 × 87.6 cm,Avant Gallery,...,#4E443A,#857C6B,#191816,#B8B3A0,#4E443A,#857C6B,69.0,176.0,185.0,36.0
4,painting52,painting,https://www.artsy.net/artwork/miaz-brothers-a-...,Miaz Brothers,https://www.artsy.net/artist/miaz-brothers,A Promising Young Man,2022,Acrylic on canvas,48 × 35 4/5 in | 122 × 91 cm,Maddox Gallery,...,#2C2D27,#A47C41,#13140F,#5B4329,#2C2D27,#A47C41,113.0,208.0,71.0,238.0


In [28]:
df.to_csv("FinalDataSet_with_colorgroups.csv")

In [27]:
len(df)

2527